In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.covariance import EmpiricalCovariance

In [2]:
def load_nifti(file_path):
    # Load the NIfTI image
    nii_image = nib.load(file_path)

    # Access the image data as a Numpy array
    data_array = nii_image.get_fdata()

    return data_array, nii_image

def show_nifti(file_data, slice=20):
    plt.imshow(file_data[:, :, slice], cmap='gray')  # Display a single slice (change the slice index as needed)
    plt.title('NIfTI Image')
    plt.colorbar()
    plt.show()

In [3]:
# subject_id  = str(1)

# data_folder_path = os.path.join(os.getcwd(), f'../Lab 1/P2_data/{subject_id}/results_60_0.0001/')

In [4]:
# file_name_ss        = 'output_ss_60_0.0001.nii'
# file_name_merged    = 'merged_segmentation.nii'

# nifti_file_ss = os.path.join(data_folder_path, file_name_ss)
# nifti_file_merged = os.path.join(data_folder_path, file_name_merged)


In [5]:
# nifti_vol_ss, _ = load_nifti(nifti_file_ss)
# show_nifti(nifti_vol_ss) # ss

In [6]:
# nifti_vol_merged, nifti_vol_merged_nii = load_nifti(nifti_file_merged)
# show_nifti(nifti_vol_merged) # merged

In [7]:
# # Load your NIfTI volume (assuming you have a function called load_nifti that loads the data)
# # nifti_vol = load_nifti(nifti_file)

# # Reshape the volume to a 2D array for clustering
# # The shape (240, 240, 48) will be reshaped to (240 * 240 * 48, 1)
# data = selected_tissue.reshape(-1, 1)

# # Perform k-means clustering with k=3
# kmeans = KMeans(n_clusters=4, random_state=0, n_init='auto')
# kmeans.fit(data)

# # Get the cluster labels and reshape them back to the original volume shape
# # cluster_labels = kmeans.labels_.reshape(nifti_vol_ss.shape)


#### Class code notes

In [10]:
# Functions implementation

def remove_black_bg(subject_id=1, labels_gt_file='LabelsForTesting.nii', skull_stripped_file='output_ss_60_0.0001.nii'):
    '''Removes the black background from the skull stripped volume and returns a 1D array of the voxel intensities \
        of the pixels that falls in the True region of the mask, for GM, WM, and CSF.'''
    
    # selecting paths
    data_folder_path    = os.path.join(os.getcwd(), f'../Lab 1/P2_data/{subject_id}')
    labels_gt           = os.path.join(data_folder_path, labels_gt_file)
    skull_stripped_path = os.path.join(data_folder_path, f'results_60_0.0001/{skull_stripped_file}')

    # load the nifti files
    labels_nifti, _ = load_nifti(labels_gt)
    skull_stripped, _    = load_nifti(skull_stripped_path)

    # Selecting the tissues based on their labels
    labels_nifti_csf = labels_nifti == 1
    labels_nifti_gm  = labels_nifti == 2
    labels_nifti_wm  = labels_nifti == 3

    # Adding the tissues togather to make a single file for the 3 tissues
    tissue_labels    = labels_nifti_csf + labels_nifti_gm + labels_nifti_wm

    # selecting the voxel values from the skull stripped
    selected_tissue = skull_stripped[tissue_labels]

    # The true mask labels count must equal to the number of voxels we segmented
    # np.count_nonzero(tissue_labels) returns the sum of pixel values that are True, the count should be equal to the number
    # of pixels in the selected tissue array
    assert np.count_nonzero(tissue_labels) == selected_tissue.shape[0], 'Error while removing the black background.'

    return selected_tissue  

def initialize_parameters(random=False):
    cluster_means = []
    cluster_covariance_matrices = []

    if not random:
        # selecting the tissues for T1 and T2-Flair
        selected_tissue = remove_black_bg(subject_id=1, labels_gt_file='LabelsForTesting.nii', skull_stripped_file='output_ss_60_0.0001.nii')

        # Reshape the volume to a 2D array for clustering
        # The shape (240, 240, 48) will be reshaped to (240 * 240 * 48, 1)
        data = selected_tissue.reshape(-1, 1)

        # Perform k-means clustering with k=3
        kmeans = KMeans(n_clusters=3, random_state=0, n_init='auto')
        kmeans.fit(data)

        # Get the cluster labels
        cluster_labels = kmeans.labels_

        # Calculate the means and covariance matrices for each cluster
        for cluster_id in range(3):  # Assuming k=3
            cluster_data = data[cluster_labels == cluster_id]
            
            # Calculate the mean for this cluster
            cluster_mean = np.mean(cluster_data)
            cluster_means.append(cluster_mean)
            
            # Calculate the empirical covariance matrix for this cluster
            covariance_matrix = EmpiricalCovariance().fit(cluster_data)
            cluster_covariance_matrices.append(covariance_matrix.covariance_)

    else:
        # random initialization of the model parameters
        pass

    return cluster_means, cluster_covariance_matrices
    
    

In [11]:
# initialization, either by random initialization or k-means
cluster_means, cluster_covariance_matrices = initialize_parameters()

print(cluster_means, cluster_covariance_matrices)

# loop over all iterations
    # loop over all k-clusters
        # compute expectation step:
            # -
            # -

        # compute the log-likelihood stopping condition

        # compute maximization step:
            # - 
            # - 



[135.24932227159948, 44.84391557036284, 207.950837856822] [array([[491.1296015]]), array([[634.64578177]]), array([[291.68290307]])]
